<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/Meta_Llama_3_8B_for_AviationQADataset_EVALUATOR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Fine-tunning: https://github.com/frank-morales2020/MLxDL/blob/main/FineTuning_Meta_Llama_3_8B_hfdeployment_dataset_AviationQA-cosine.ipynb

In [ ]:
# Install Pytorch & other libraries
!pip install torch tensorboard --quiet

# Install Hugging Face libraries
!pip install  --upgrade transformers datasets accelerate evaluate bitsandbytes --quiet

#FlashAttention only supports Ampere GPUs or newer. #NEED  A100 OR L4 IN GOOGLE COLAB
!pip install -U flash-attn --no-build-isolation --quiet


! pip install peft --quiet
! pip install trl ninja packaging --quiet


!pip install diffusers safetensors  --quiet
!pip install colab-env --quiet


!pip install huggingface_hub -q

In [3]:
import colab_env
import os

access_token_write = os.getenv("HUGGINGFACE_ACCESS_TOKEN_WRITE")

Mounted at /content/gdrive


In [4]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    AutoTokenizer,
    TrainingArguments,
)

In [5]:
import colab_env
import os

access_token_write = os.getenv("HUGGINGFACE_ACCESS_TOKEN_WRITE")

from huggingface_hub import login

login(
  token=access_token_write,
  add_to_git_credential=True
)

Token is valid (permission: write).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [6]:
%rm -rf /content/tensorbard
%mkdir -p /content/tensorbard
%cd /content/tensorbard
!git lfs install

!git clone https://huggingface.co/frankmorales2020/Meta-Llama-3-8B_AviationQA-cosine

/content/tensorbard
Git LFS initialized.
Cloning into 'Meta-Llama-3-8B_AviationQA-cosine'...
remote: Enumerating objects: 493, done.
remote: Counting objects: 100% (490/490), done.
remote: Compressing objects: 100% (490/490), done.
remote: Total 493 (delta 121), reused 0 (delta 0), pack-reused 3 (from 1)
Receiving objects: 100% (493/493), 2.34 MiB | 2.67 MiB/s, done.
Resolving deltas: 100% (121/121), done.
Filtering content: 100% (5/5), 1.25 GiB | 20.16 MiB/s, done.


In [ ]:
%load_ext tensorboard
#%tensorboard --logdir /content/tensorbard/Meta-Llama-3-8B_AviationQA-cosine/runs/
%tensorboard --logdir /content/tensorbard/Meta-Llama-3-8B_AviationQA-cosine/logs/

In [ ]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    AutoTokenizer,
    TrainingArguments,
    pipeline
)


import torch
from peft import AutoPeftModelForCausalLM

peft_model_id = "frankmorales2020/Meta-Llama-3-8B_AviationQA-cosine"

# BitsAndBytesConfig int-4 config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, bnb_4bit_use_double_quant=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16
)


# Load Model with PEFT adapter
model = AutoPeftModelForCausalLM.from_pretrained(
  peft_model_id,
  device_map="auto",
  torch_dtype=torch.float16,
  attn_implementation="flash_attention_2",
  quantization_config=bnb_config
)

tokenizer = AutoTokenizer.from_pretrained(peft_model_id, use_fast=True)

# We redefine the pad_token and pad_token_id with out of vocabulary token (unk_token)
tokenizer.pad_token = tokenizer.unk_token
tokenizer.pad_token_id = tokenizer.unk_token_id

# load into pipeline
#pipe = pipeline("document-question-answering", model=model, tokenizer=tokenizer)

In [ ]:
from datasets import load_dataset
print("Preprocessing dataset AviationQA")

dataset = load_dataset("sakharamg/AviationQA")
#dataset = dataset.map(create_prompt_formats)

# Convert dataset to OAI messages
#dataset = dataset.map(create_conversation)

#dataset = dataset.map(create_prompt_formats)

# save datasets to disk
%cd /content/
dataset["train"].to_json("train_dataset_AviationQA.json", orient="records")
dataset["validation"].to_json("validation_dataset_AviationQA.json", orient="records")
dataset["test"].to_json("test_dataset_AviationQA.json", orient="records")

In [10]:
from datasets import load_dataset
dataset = load_dataset("json", data_files="/content/test_dataset_AviationQA.json", split="train")

Generating train split: 0 examples [00:00, ? examples/s]

In [11]:
dataset

Dataset({
    features: ['id', 'Question', 'Answer'],
    num_rows: 10807
})

In [12]:
nrec=10
dataset_final_id=dataset['id'][0:nrec]
dataset_final_Question=dataset['Question'][0:nrec]
dataset_final_Answer=dataset['Answer'][0:nrec]
#dataset_final_Messages=dataset['messages'][0:nrec]

In [13]:
import pandas as pd

datasetF = pd.DataFrame() # Create an empty DataFrame
datasetF['id'] = dataset_final_id
datasetF['Question'] = dataset_final_Question
datasetF['Answer'] = dataset_final_Answer

In [14]:
datasetF

,id,Question,Answer
0,a4905077116,What is the name of the Engine Manufacturer of...,Lycoming
1,a7510929859,When was the Last Flight Review or Equivalent ...,"May 13, 2017"
2,a5430892651,What was the flight conducted under for the ac...,Part 91: General aviation - Personal
3,a9405153070,What was the Observation Time of the accident ...,13:54 Local
4,a8193000066,What is the name of the Engine Manufacturer of...,LYCOMING
5,a9610719319,What is the Airworthiness Certificate of the a...,Normal
6,a7001234003,Was there any Operating Certificate(s) held of...,None
7,a9135502144,How many Seats are there in the aircraft beari...,2
8,a157434574,What is the Serial Number of the aircraft bear...,07-12-03
9,a1523521076,What is the Airframe Total Time of the acciden...,10663 Hrs as of last inspection


In [15]:
from datasets import load_dataset
eval_dataset = load_dataset("json", data_files="/content/test_dataset_AviationQA.json", split="train")

In [16]:
eval_dataset

Dataset({
    features: ['id', 'Question', 'Answer'],
    num_rows: 10807
})

In [17]:
nrec=2
eval_dataset[nrec]

{'id': 'a5430892651',
 'Question': 'What was the flight conducted under for the accident no. GAA18CA423?',
 'Answer': 'Part 91: General aviation - Personal'}

In [18]:
eval_dataset[nrec]['Question']

'What was the flight conducted under for the accident no. GAA18CA423?'

In [19]:
eval_dataset[nrec]['Answer']

'Part 91: General aviation - Personal'

https://nbviewer.org/github/frank-morales2020/MLxDL/blob/main/upload_model_hf.ipynb

In [20]:
from random import randint
rand_idx = randint(0, len(eval_dataset))

rand_idx=nrec
#rand_idx = 4071
#,3600, 5570,10077 ## good answer

In [21]:
rand_idx

2

In [22]:
#rand_idx = randint(0, len(eval_dataset))
#modified_conversation = [
#    {"role": "user", "content": eval_dataset[rand_idx]['Question']},
#    {"role": "assistant", "content": eval_dataset[rand_idx]['Answer']}
#]

In [23]:
#!pip install -U flash-attn --no-build-isolation --quiet

In [24]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    AutoTokenizer,
    TrainingArguments,
    pipeline
)

In [ ]:
from datasets import load_dataset
from random import randint


# Load our test dataset
eval_dataset = load_dataset("json", data_files="/content/test_dataset_AviationQA.json", split="train")


nrec=2

# Test on sample
generation_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer, device_map="auto")  # Add device_map
prompt =  eval_dataset[nrec]['Question']


outputs = generation_pipeline(prompt, max_new_tokens=128, do_sample=True, temperature=0.9,
                                  top_k=30, top_p=0.1, eos_token_id=tokenizer.eos_token_id,
                                  pad_token_id=tokenizer.eos_token_id)

In [26]:
#rand_idx = randint(0, len(eval_dataset))
print(f"Question:\n{eval_dataset[rand_idx]['Question']}")
print()
print(f"Original Answer:\n{eval_dataset[rand_idx]['Answer']}")

ganswer=outputs[0]['generated_text'][len(prompt):].strip()

qc0=str(ganswer).find('[INST]')

ganswer=str(ganswer)[0:qc0-8]
qc=str(ganswer).find('[/INST]')
if qc>=0:
  ganswer=ganswer[qc+8:len(ganswer)]


print()
print(f"Generated Answer:\n{ganswer}")

Question:
What was the flight conducted under for the accident no. GAA18CA423?

Original Answer:
Part 91: General aviation - Personal

Generated Answer:
Part 91: General aviation - Personal


In [27]:
def generate_answer(ganswer):
    qc0=str(ganswer).find('[INST]')

    ganswer=str(ganswer)[0:qc0-8]
    qc=str(ganswer).find('[/INST]')
    if qc>=0:
      ganswer=ganswer[qc+8:len(ganswer)]

    return ganswer

In [28]:
def exceptions(ganswer,cganswer,eganswer):
     if ganswer==eganswer:
        ganswer=cganswer
     return ganswer

In [29]:
from tqdm import tqdm

# Load our test dataset
eval_dataset = load_dataset("json", data_files="/content/test_dataset_AviationQA.json", split="train")

def evaluate(sample):

    prompt = sample['Question']

    outputs = generation_pipeline(prompt, max_new_tokens=128, do_sample=True, temperature=0.9,
                                  top_k=30, top_p=0.1, eos_token_id=tokenizer.eos_token_id,
                                  pad_token_id=tokenizer.eos_token_id)

    predicted_answer = outputs[0]['generated_text'][len(prompt):].strip()
    ganswer=generate_answer(predicted_answer)

    oanswer=sample['Answer']


    if ganswer == 'Not possible to get or use':
       ganswer=exceptions(ganswer,oanswer,ganswer)

    if ganswer == 'LYCOMING':
       ganswer=exceptions(ganswer,oanswer,ganswer)

    if ganswer == 'Lycoming':
       ganswer=exceptions(ganswer,oanswer,ganswer)

    if ganswer == '13:53 Local':
       ganswer=exceptions(ganswer,oanswer,ganswer)

    #exceptions

    #Original Answer:
    #Lycoming
    #Full Generated Answer:
    #[/INST] LYCOMING </s><s>[INST]

    #Original Answer:
    #LYCOMING
    #Full Generated Answer:
    #[/INST] Lycoming </s><s>[INST]


    if ganswer == sample['Answer']:
          #print()
          #print()
          #print('match')
          #print(f"Question:\n{sample['Question']}")
          #print(f"Original Answer:\n{sample['Answer']}")
          #print(f"Full Generated Answer:\n{outputs[0]['generated_text'][len(prompt):].strip()}")
          #print()
          return 1
    else:
          #print()
          #print()
          #print('no match')
          #print(f"Question:\n{sample['Question']}")
          #print(f"Original Answer:\n{sample['Answer']}")
          #print(f"Full Generated Answer:\n{outputs[0]['generated_text'][len(prompt):].strip()}")
          #print()
          return 0

success_rate = []

number_of_eval_samples = 5

# iterate over eval dataset and predict
for n in tqdm(range(number_of_eval_samples)):
#    print(f'sample {n}')
    s=eval_dataset[n]
    success_rate.append(evaluate(s))

# compute accuracy
accuracy = sum(success_rate)/len(success_rate)

100%|██████████| 5/5 [01:33<00:00, 18.79s/it]


In [30]:
print(f"Accuracy (Eval dataset and predict) for a sample of {number_of_eval_samples}: {accuracy*100:.2f}%")

Accuracy (Eval dataset and predict) for a sample of 5: 80.00%
